C. Compile a several demographic variables (ideally at a census tract or similar level of granularity) and overlay onto the map from Part B. These variables at a minimum include:

a) income,

b) language spoken (or geometries),

c) population density (to normalize for the amount of retail shops per 100k people),

d) Digital Divide Index or some other metric of internet connectivity

DELIVERABLE: A mapped visualization of the above variables with the retail shop locations 
    from Part B, the corresponding shapefiles for the above variables, and an updated spreadsheet that pairs these variables to the spreadsheet from Part B.

In [270]:
import pandas as pd
import geopandas as gp
import requests
from tqdm import tqdm
from shapely.geometry import shape 
import pyproj
from shapely.ops import transform
from functools import partial

from config import QKEY

pd.set_option('display.max_colwidth', 20)
pd.set_option('display.max_row', 2000)

In [271]:
df = pd.read_csv('files/tire_shops.csv')

# Income

In [272]:
def get_income_and_geo_id(row):
    url = f'https://quantmaps.com/api/locations/detail/2019/TRACT/?coords={row.lat},{row.lng}'
    response = requests.get(url, headers={"Authorization": QKEY}, params = {"fields":"median_household_income,total_population,geom"})
    response_dict = response.json()
    return response_dict['geo_id'], response_dict['median_household_income'], response_dict['total_population'], response_dict['geom']

In [273]:
for n, row in tqdm(df.iterrows(), total=len(df)):
    geo_id, income, total_pop, geometries = get_income_and_geo_id(row)
    df.loc[n, 'geo_id'] = geo_id
    df.loc[n, 'income'] = income
    df.loc[n, 'total_pop'] = total_pop
    df.loc[n, 'geom'] = shape(geometries)

100%|██████████████████████████████████████████████████████████████████████████████████| 89/89 [00:26<00:00,  3.39it/s]


# Language Spoken: percentage of non-English speakers

In [274]:
language_df = pd.read_csv('ACSST5Y2018.S1602_data_with_overlays_2020-10-31T111916.csv')
headers_df = pd.read_csv('ACSST5Y2018.S1602_metadata_2020-10-31T111916.csv')
language_df = language_df[['GEO_ID', 'S1602_C04_001E']][1:]
language_df["GEO_ID"] = language_df.GEO_ID.apply(lambda x: x[9:])
language_df = language_df.rename(columns={'GEO_ID':'geo_id', "S1602_C04_001E":headers_df.loc[headers_df['GEO_ID']=='S1602_C04_001E'].id.item()})
language_df.head()

,geo_id,Estimate!!Percent limited English-speaking households!!All households
1,06001400100,1.5
2,06001400200,0.0
3,06001400300,8.1
4,06001400400,1.2
5,06001400500,1.4


In [275]:
df = df.merge(language_df, on='geo_id', how='left')

# Internet Connectivity

In [276]:
internet_df = pd.read_csv('ACSST5Y2018.S2801_data_with_overlays_2020-10-31T111552.csv')
headers_df = pd.read_csv('ACSST5Y2018.S2801_metadata_2020-10-31T111552.csv')
internet_df = internet_df[['GEO_ID', 'S2801_C02_012E']][1:]
internet_df["GEO_ID"] = internet_df.GEO_ID.apply(lambda x: x[9:])
internet_df = internet_df.rename(columns={'GEO_ID':'geo_id', "S2801_C02_012E":headers_df.loc[headers_df['GEO_ID']=='S2801_C02_012E'].id.item()})
internet_df.head()

,geo_id,Estimate!!Percent!!TYPE OF INTERNET SUBSCRIPTIONS!!With an Internet subscription
1,06001400100,96.3
2,06001400200,96.7
3,06001400300,89.9
4,06001400400,94.6
5,06001400500,95.1


In [277]:
df = df.merge(internet_df, on='geo_id', how='left')

# Population Density

In [278]:
def get_pop_density(row):
    proj = partial(pyproj.transform, pyproj.Proj(init='epsg:4326'), pyproj.Proj(init='epsg:3857'))
    projected_area = transform(proj, row.geom).area
    return round(row.total_pop/(projected_area*0.00000038610)*1.66)

In [279]:
df['pop_density'] = df.apply(get_pop_density, axis=1)

# Make GEO df and Clean

In [280]:
# df = pd.read_csv('demographics.csv')
# df = df.drop('Unnamed: 0', axis=1)
df = df.rename(columns={
    "Estimate!!Percent limited English-speaking households!!All households": "% Non-English",
    "Estimate!!Percent!!TYPE OF INTERNET SUBSCRIPTIONS!!With an Internet subscription": "% Internet",
    "pop_density": "Pop Density",
    "geo_id": "Census Tract ID",
    'income': 'Income',
    'geom': 'geometry'
})

gdf = gp.GeoDataFrame(df[['Census Tract ID', 
                         '% Non-English', 
                         '% Internet',
                         'Pop Density',
                         'Income',
                         'geometry']])

In [282]:
df.to_csv('shops.csv', index=False)
gdf.to_file('tracts.shp')

c:\users\art perkitny\anaconda3\lib\site-packages\geopandas\io\file.py:108: FionaDeprecationWarning: Use fiona.Env() instead.
  with fiona.drivers():


In [283]:
gdf.dtypes

Census Tract ID     object
% Non-English       object
% Internet          object
Pop Density          int64
Income             float64
geometry            object
dtype: object

In [284]:
df

,name,address,phone_number,lat,lng,Sells Used Tires?,Method to Confirm,Link,Notes,Census Tract ID,Income,total_pop,geometry,% Non-English,% Internet,Pop Density
0,1-800EveryRim OE...,12078 Florence A...,56269201090,33.9347,-118.0745,True,Review,https://www.yelp...,NaN,06037502802,34907.0,3081.0,POLYGON ((-118.0...,18.2,74.4,2356
1,A & V Tires,1610 Oxnard Blvd...,80538511920,34.2168,-119.1771,True,Review,https://www.yelp...,NaN,06111003100,102358.0,14669.0,POLYGON ((-119.1...,4.8,89.9,10761
2,Aal STAR TIRE,31162 San Clemen...,51048952180,37.6090,-122.0522,True,Call,NaN,NaN,06001438203,71607.0,4118.0,POLYGON ((-122.0...,13.0,89.3,2095
3,Able Tire & Brak...,7090 Redwood Blv...,41589741250,38.1004,-122.5669,True,Review,https://www.yelp...,NaN,06041102203,67839.0,5736.0,POLYGON ((-122.5...,6.5,84.0,5578
4,Arcata Used Rire...,437 G Street Arc...,70782279090,40.8652,-124.0876,True,Review,https://www.yelp...,NaN,06023001000,24250.0,6204.0,POLYGON ((-124.0...,3.4,90.1,2777
5,Balswick Tire Sh...,417 N. Golden St...,20963458820,37.4970,-120.8485,True,Review,https://www.yelp...,NaN,06099003904,49618.0,4655.0,POLYGON ((-120.8...,12.3,73.0,3448
6,Bob's Tire Center,2300 Esplanade C...,53089584730,39.7529,-121.8572,True,Review,https://www.yelp...,NaN,06007000300,28513.0,4784.0,POLYGON ((-121.8...,2.6,83.0,4616
7,Bruce's Tire Inc.,2420 Prune Ave. ...,51058014410,37.5062,-121.9385,True,Review,https://www.yelp...,NaN,06001441503,164118.0,6834.0,POLYGON ((-122.1...,9.9,98.7,205
8,Buds Tire,8651 Indiana Ave...,95135113190,33.9230,-117.4264,True,Review,https://www.yelp...,NaN,06065031704,72850.0,5584.0,POLYGON ((-117.4...,9.8,77.8,6320
9,Bymar Tire & Brake,25631 Taladro Ci...,94976873810,33.6195,-117.6835,True,Review,https://www.yelp...,NaN,06059032028,91694.0,3123.0,POLYGON ((-117.6...,10.8,90.7,4476
